In [1]:
import os
import collections
import tensorflow as tf
import numpy as np

In [2]:
f = open('./poetry/poetry.txt')
n = 0
poetries = []
for line in f:
    line = line.decode('UTF8').strip('\n')
    poetries.append(line)

In [3]:
all_words = []
for p in poetries:
    all_words += [word for word in p]
counter = collections.Counter(all_words)

counter = sorted(counter.items(), key = lambda x:-x[1])

idx_to_words, _ = zip(*counter)
idx_to_words = idx_to_words[:len(idx_to_words)] + (' '.decode('UTF8'),)

In [4]:
word_to_idx = dict(zip(idx_to_words, range(len(idx_to_words))))
poetry_vector = [ list(map(lambda word: word_to_idx.get(word,len(idx_to_words)), poetry)) for poetry in poetries]
def data_batch(start, end):
    batches = []
    for i in xrange(start,end):
        batches.append(poetry_vector[i])
    
    length = max(map(len, batches))
    
    xdata = np.full([end - start, length], word_to_idx[' '], np.int32)
    for i in range(end - start):
        xdata[i, :len(batches[i])] = batches[i]
    
    ydata = np.copy(xdata)
    ydata[:,:-1] = xdata[:,1:]
    print '-----------',xdata.shape,ydata.shape
    return xdata, ydata

In [5]:
tf.reset_default_graph()
batch_size = 8
vocab_size = len(idx_to_words)
hidden_size = 16
#num_steps = 10
#lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_size)
#cell = tf.contrib.rnn.MultiRNNCell([lstm_cell] * 2)
cell = tf.contrib.rnn.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(hidden_size) for _ in range(2)])

input_data = tf.placeholder(tf.int32, [None, None])#第一个维度是batch_size,第二个维度是num_steps
targets = tf.placeholder(tf.int32, [None, None])#跟上面一致

embedding = tf.get_variable(name='embedding', shape=[len(idx_to_words), hidden_size])

inputs = tf.nn.embedding_lookup(embedding, input_data)


initial_state = cell.zero_state(batch_size, tf.float32)

state = initial_state

#with tf.variable_scope('RNN'):
#    for time_step in range(dataset.sample_len):
#        if time_step > 0: tf.get_variable_scope().reuse_variables()
#        cell_output, state = cell(inputs[:,time_step,:], state)
#        outputs.append(cell_output)
        
        
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state)
#output = tf.reshape(tf.concat(outputs, 1), [-1, hidden_size])
output = tf.reshape(outputs, [-1, hidden_size])


weight = tf.get_variable('weight', [hidden_size, vocab_size])
bias = tf.get_variable('bias', [vocab_size])
logits = tf.matmul(output, weight) + bias

loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(targets,[-1])],
                                                          [tf.ones_like(tf.reshape(targets,[-1]), dtype = tf.float32)])
cost = tf.reduce_sum(loss) / batch_size

#final_state = state

trainable_variables = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(cost, trainable_variables), clip_norm=5)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0)
train_op = optimizer.apply_gradients(zip(grads, trainable_variables))

In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    epoch = 10
    for i in xrange(epoch):
        x, y = data_batch(i,i+batch_size)
        sess.run(train_op, feed_dict={input_data:x, targets: y})
    
    #saver.save(sess,'./model/model.ckpt')

----------- (8, 534) (8, 534)
(4272, 7648)
(8, 534)
----------- (8, 534) (8, 534)
(4272, 7648)
(8, 534)
----------- (8, 534) (8, 534)
(4272, 7648)
(8, 534)
----------- (8, 534) (8, 534)
(4272, 7648)
(8, 534)
----------- (8, 534) (8, 534)
(4272, 7648)
(8, 534)
----------- (8, 534) (8, 534)
(4272, 7648)
(8, 534)
----------- (8, 534) (8, 534)
(4272, 7648)
(8, 534)
----------- (8, 534) (8, 534)
(4272, 7648)
(8, 534)
----------- (8, 246) (8, 246)
(1968, 7648)
(8, 246)
----------- (8, 246) (8, 246)
(1968, 7648)
(8, 246)


In [22]:
logits.shape

TensorShape([Dimension(None), Dimension(7648)])

In [23]:
targets.shape

TensorShape([Dimension(None), Dimension(None)])